In [1]:
import numpy as np
import pandas as pd
# import  jupyter_pandas.helper as helper


In [2]:
pd.set_option('display.max_columns',None)
df = pd.read_excel("Data Scientist - Project.xlsx",header=0)

### ADD INDEX COLUMN


In [3]:
for index,name in enumerate(df["NAME"]):
    df.loc[index,'INDEX'] = index
   

 Only show the rows where the region is not europe

In [4]:
not_europe = df['HQ REGION'] != 'Europe'
df[not_europe]['HQ REGION']

12             Africa
241              Asia
250              Asia
325     North America
368              Asia
            ...      
6699    North America
7174    North America
7636    North America
7804    North America
8701             Asia
Name: HQ REGION, Length: 100, dtype: object

Show unique values of the HQ REGION column

In [5]:
df['HQ REGION'].unique()

array(['Europe', 'Africa', 'Asia', 'North America', 'South America',
       'Oceania'], dtype=object)

 Show datatypes for each column

In [6]:
df.dtypes

NAME                                                  object
PROFILE URL                                           object
WEBSITE                                               object
HQ REGION                                             object
HQ COUNTRY                                            object
HQ CITY                                               object
INVESTORS                                             object
EACH INVESTOR TYPES                                   object
LEAD INVESTORS                                       float64
TOTAL FUNDING (EUR M)                                float64
TOTAL FUNDING (USD M)                                float64
LAST ROUND                                            object
LAST FUNDING                                         float64
LAST FUNDING DATE                                     object
FIRST FUNDING DATE                                    object
SEED YEAR                                            float64
REVENUE MODEL           

Convert all __Each Round Amount__ cells to a list of numbers and add the result as a new column __Each Round Amount as List__ to the dataframe

In [7]:
#function that takes in a value from a cell (string) and converts it to a list of numbers
def convert_to_list_of_numbers(cellValue):
        cellValueAsList =  cellValue.split(';')
        return [float(x) if x != 'n/a' else 0 for x in cellValueAsList]
# Convert all cells of the "each round amount" column to a list with the function above
# if there is something to convert, otherwise create a list with just a 0 in it for that cell
df['EACH ROUND AMOUNT AS LIST'] = [convert_to_list_of_numbers(cell) if isinstance(cell, str) else [0] for cell in df['EACH ROUND AMOUNT']]
df['EACH ROUND AMOUNT AS LIST']


0                                  [0, 0, 0]
1                              [0, 1.7, 0.6]
2       [11.5, 0, 0.9, 2.3, 0.05, 2.5, 0.05]
3                                     [0, 0]
4                                        [0]
                        ...                 
8889                                     [0]
8890                                     [0]
8891                                     [0]
8892                                     [0]
8893                                     [0]
Name: EACH ROUND AMOUNT AS LIST, Length: 8894, dtype: object

Using the new column __Each Round Amount as List__ to create yet another column __Each Round Amount Sum__

In [8]:
df['EACH ROUND AMOUNT SUM'] = [sum(cell)for cell in df['EACH ROUND AMOUNT AS LIST']]

By replacing non existing values with 0 we run into issues when using functions like mean() on a column

In [9]:
df['EACH ROUND AMOUNT SUM']

0        0.0
1        2.3
2       17.3
3        0.0
4        0.0
        ... 
8889     0.0
8890     0.0
8891     0.0
8892     0.0
8893     0.0
Name: EACH ROUND AMOUNT SUM, Length: 8894, dtype: float64

We get a value but its **not correct**. If we want the correct mean we can replace all zeros with null and calculate
the mean again and we get the **correct** result because the mean function has a default parameter of *skipna=True*

In [10]:

df['EACH ROUND AMOUNT SUM NULL'] = [None if cell == 0 else cell for cell in df['EACH ROUND AMOUNT SUM']]


df['EACH ROUND AMOUNT SUM NULL'].mean()


19.743192612137204

Now we try to convert the **LAST FUNDING DATE** from a String to Date dataobject, so we can filter and compare dates correctly. First we look at what we have:

In [11]:
df["LAST FUNDING DATE"].unique()






array(['jan/2022', nan, 'dec/2021', 'nov/2021', 'oct/2021', 'sep/2021',
       'aug/2021', 'jul/2021', 'jun/2021', 'may/2021', 'apr/2021',
       'mar/2021', 'feb/2021', 'jan/2021', 'dec/2020', 'nov/2020',
       'oct/2020', 'sep/2020', 'aug/2020', 'jul/2020', 'jun/2020',
       'may/2020', 'apr/2020', 'mar/2020', 'feb/2020', 'jan/2020',
       'dec/2019', 2020, 'nov/2019', 'oct/2019', 'sep/2019', 'aug/2019',
       'jul/2019', 'jun/2019', 'may/2019', 'apr/2019', 'mar/2019',
       'feb/2019', 'jan/2019', 'dec/2018', 2019, 'nov/2018', 'oct/2018',
       'sep/2018', 'aug/2018', 'jul/2018', 'jun/2018', 'may/2018',
       'apr/2018', 'mar/2018', 'feb/2018', 'jan/2018', 'dec/2017', 2018,
       'nov/2017', 'oct/2017', 'sep/2017', 'aug/2017', 'jul/2017',
       'jun/2017', 'may/2017', 'apr/2017', 'mar/2017', 'feb/2017',
       'jan/2017', 'dec/2016', 2017, 'nov/2016', 'oct/2016', 'sep/2016',
       'aug/2016', 'jul/2016', 'jun/2016', 'may/2016', 'apr/2016',
       'mar/2016', 'feb/2016', 'j

We have **NaN**, **Numbers** like 2010 and **Strings** with a correct format like nov/2011. Lets clear this up by setting NaN to a "jan/1970" String and the numbers to "jan/number"

In [12]:
df['LAST FUNDING DATE'] = df['LAST FUNDING DATE'].apply(lambda cell: cell if isinstance(cell, str)
                                                        else f'jan/{cell}' if cell > 0
                                                        else None)

df['LAST FUNDING DATE'] = pd.to_datetime(df['LAST FUNDING DATE'])

df['LAST FUNDING DATE']





0      2022-01-01
1      2022-01-01
2      2022-01-01
3      2022-01-01
4             NaT
          ...    
8889          NaT
8890          NaT
8891          NaT
8892          NaT
8893          NaT
Name: LAST FUNDING DATE, Length: 8894, dtype: datetime64[ns]

Now we can filter the column for Dates like this:

In [13]:
after_2018 = df['LAST FUNDING DATE'] > pd.Timestamp(2018,1,1) 
no_date_available = df['LAST FUNDING DATE'].apply(lambda row: True if str(row) == 'NaT' else False)
df[no_date_available | after_2018]['LAST FUNDING DATE']


0      2022-01-01
1      2022-01-01
2      2022-01-01
3      2022-01-01
4             NaT
          ...    
8889          NaT
8890          NaT
8891          NaT
8892          NaT
8893          NaT
Name: LAST FUNDING DATE, Length: 5496, dtype: datetime64[ns]

With datetime datatype its easy to handle Dates --> [Pandas Timestamp Reference](https://pandas.pydata.org/docs/reference/api/pandas.Timestamp.html)

In [14]:
df[after_2018]['LAST FUNDING DATE'][0].year


2022

Create a boolean column for every Industry in the Dataset and set the appropiate columns to True for every StartUp

In [15]:
#Transformation, damit man Industries später besser algorithmisch nutzen kann (one-hot-mapping)
industries = {}
#alle unique industries der Tabelle nehmen und in eine Map/Dictonary einfügen
for row in df['INDUSTRIES']:
   if isinstance(row,str):
       industries_of_current_row = row.split(";")
       for industry in industries_of_current_row:
           industries[industry.upper()] = industry.upper()
#neue Spalte im df anlegen für jede industry und erstmal mit false initialisieren
# im Format IS_VARIABLE_INDUSTRY
for key in industries:
    df[f'IS_{key}_INDUSTRY'] = False
    

 # Wir iterieren wieder durch die Industry Spalte, brauchen aber jetzt zusätzlich den aktuellen Index (enumerate)
 # und setzen für jede die passende IS_VARIABLE_INDUSTRY Spalte auf true               
for rowindex,row in enumerate(df['INDUSTRIES']):
    if isinstance(row, str):
       industries_of_current_row = row.split(";")
       for industry in industries_of_current_row:
           df.loc[rowindex,f'IS_{industry.upper()}_INDUSTRY'] = True
          

df['IS_ROBOTICS_INDUSTRY']      

0       False
1       False
2        True
3       False
4       False
        ...  
8889    False
8890    False
8891    False
8892    False
8893    False
Name: IS_ROBOTICS_INDUSTRY, Length: 8894, dtype: bool

Create Average Valuation Column

In [16]:
#Function to set average value from valuation interval (min + max)/2
def avg_valuation(cellValue):
    val = cellValue.split('-')
    return ((float(val[1]) + float(val[0]))/2)/10**6


#Parse both valuation columns and create maximum valuation column
valuationColumns = ['VALUATION', 'VALUATION (EUR)']
for column in valuationColumns:
    df.loc[:, column + ' AVG'] = df.loc[:, column]
    df.loc[:, column + ' AVG'] = [avg_valuation(cell) if isinstance(cell, str)
                                           else cell for cell in df['VALUATION AVG']]
df["VALUATION AVG"]

0        NaN
1        8.5
2       12.5
3        NaN
4        NaN
        ... 
8889     NaN
8890     NaN
8891     NaN
8892     NaN
8893     NaN
Name: VALUATION AVG, Length: 8894, dtype: float64

Creating a new Object from combining the EACH ROUND TYPE, AMOUNT, DATE, INVESTORS Column. This will help to create new columns later on

In [17]:
import requests
currencyMap = {}
# Get all distinct Currencies in the Dataset and set them with a default value
for row in df['EACH ROUND CURRENCY']:
    if isinstance(row, str):
        for currency in row.split(';'):
            if(currency != 'n/a'):
                currencyMap[currency] = 0.0
            else:
                currencyMap[currency] = 1

#Get Currency Data from API for all currencies in the currency Map
currency_request = requests.get(
    f'https://api.currencyscoop.com/v1/latest?api_key=60e7bc56b004cb3a965554c6a1b50c3d&base=usd&symbols={",".join(currencyMap.keys())}', headers={'User-agent': 'Mozilla/5.0'})

#Set the correct values in the currency Map
for currency, value in currency_request.json()['response']['rates'].items():
    currencyMap[currency] = value
each_round_data = []

#Iterate the each round Amount column and use the row index for cross-lookups in other columns
for row_index, row in enumerate(df['EACH ROUND AMOUNT']):
    #funding object which will be enriched and then added to each_round_data after a row has been processed
    funding_object = {
        "EACH ROUND AMOUNT": [],
        "EACH ROUND DATE": [],
        "EACH ROUND INVESTORS": [],
        "EACH ROUND TYPE": []
    }
    #if there are multiple amounts: split the list, get the value, divide the value by the currency rate, extract the dates and add them to the funding object
    if isinstance(row, str):
        for index, amount in enumerate(row.split(";")):
            if(amount != 'n/a'):
                funding_object['EACH ROUND AMOUNT'].insert(0,float(amount) / currencyMap[df.loc[row_index, 'EACH ROUND CURRENCY'].split(";")[index]])
            else:
                funding_object['EACH ROUND AMOUNT'].insert(0,0)
            funding_object['EACH ROUND TYPE'].insert(0, df.loc[row_index, 'EACH ROUND TYPE'].split(";")[index])
            funding_object['EACH ROUND INVESTORS'].insert(0, df.loc[row_index, 'EACH ROUND INVESTORS'].split(";")[index])
            funding_object['EACH ROUND DATE'].insert(0, pd.to_datetime(str(df.loc[row_index, 'EACH ROUND DATE'].split(";")[index]))) 
    # if there is just one amount or none, take just that value and do the same as above        
    else:
        if row > 0:
            funding_object['EACH ROUND AMOUNT'].insert(0, row / currencyMap[df.loc[row_index, 'EACH ROUND CURRENCY']])
            funding_object['EACH ROUND TYPE'].insert(0, df.loc[row_index, 'EACH ROUND TYPE'])
            funding_object['EACH ROUND INVESTORS'].insert(0, df.loc[row_index, 'EACH ROUND INVESTORS'])
            funding_object['EACH ROUND DATE'].insert(0, pd.to_datetime(str(df.loc[row_index, 'EACH ROUND DATE'])))
    each_round_data.append(funding_object)







           

Add total funding usd column to the dataframe using the each_round_data object


In [18]:
   
df["TOTAL FUNDING USD"] = [sum(funding["EACH ROUND AMOUNT"]) for funding in each_round_data]


Add column Funding from seed type using the each_round_data object

In [19]:
seed_funding = []

for data in each_round_data:
    seed_sum = 0
    for index, funding_type in enumerate(data["EACH ROUND TYPE"]):
        if funding_type == "SEED":
            seed_sum += data["EACH ROUND AMOUNT"][index]
    seed_funding.append(seed_sum)
df["FUNDING FROM SEED TYPE"] = seed_funding


Create FUNDING FROM EARLY STAGES where only valid funding types are taken into account (Svunkis Filter)

In [20]:
neglected_rounds = set(['ACQUISITION',
                         'LATE VC', 'IPO', 'MERGER', 'POST IPO DEBT', 'POST IPO EQUITY', 'GROWTH EQUITY NON VC', 'GROWTH EQUITY VC', 'POST IPO CONVERTIBLE', 'SERIES C', 'SERIES B', 'SERIES D'])


filtered_funding = []

for data in each_round_data:
    seed_sum = 0
    for index, funding_type in enumerate(data["EACH ROUND TYPE"]):
        if funding_type not in neglected_rounds:
            seed_sum += data["EACH ROUND AMOUNT"][index]
    filtered_funding.append(seed_sum)
df["FUNDING FROM EARLY STAGES"] = filtered_funding


df["FUNDING FROM EARLY STAGES"]


0        0.000000
1        2.592244
2       17.604950
3        0.000000
4        0.000000
          ...    
8889     0.000000
8890     0.000000
8891     0.000000
8892     0.000000
8893     0.000000
Name: FUNDING FROM EARLY STAGES, Length: 8894, dtype: float64

Create Column  Filtered Funding > 2M

In [21]:
df["FUNDING FROM EARLY STAGES > 2M"] = df['FUNDING FROM EARLY STAGES'].apply(
    lambda row: True if row > 2 else False)



Now we create a Column Funding up until Valuation Date

In [22]:
df["VALUATION DATE"] = df["VALUATION DATE"].apply(
    lambda row: pd.to_datetime(str(row)))


df["VALUATION DATE"]


before_valuation_funding = []

for index, data in enumerate(each_round_data):
    seed_sum = 0
    if str(df.loc[index, "VALUATION DATE"]) != "NaT":
        date_limit = df.loc[index, "VALUATION DATE"]

        for index, date in enumerate(data["EACH ROUND DATE"]):
            if str(date) != "NaT" and date <= date_limit:
                seed_sum += data["EACH ROUND AMOUNT"][index]

    before_valuation_funding.append(seed_sum)

df["FUNDING UP UNTIL VALUATION DATE"] = before_valuation_funding


df["FUNDING UP UNTIL VALUATION DATE"]

# df["FUNDING FROM EARLY STAGES"]


0       0.000000
1       2.592244
2       2.556353
3       0.000000
4       0.000000
          ...   
8889    0.000000
8890    0.000000
8891    0.000000
8892    0.000000
8893    0.000000
Name: FUNDING UP UNTIL VALUATION DATE, Length: 8894, dtype: float64

Next up we create the Column REVENUE PER YEAR where only the years are taken into account with revenue data available and AVG REVENUE GROWTH with the same restrictions and

In [23]:


revenue_as_list = df["REVENUE (2016,2017,2018,2019,2020,2021,2022,2023)"].apply(lambda cell: convert_to_list_of_numbers(cell))

def calculate_avg_revenue(cell):
    revenue_count = 0.0
    revenue_sum = 0.0
    for index, revenue in enumerate(cell):
        if revenue > 0:
            revenue_count += 1
            revenue_sum += revenue
    if revenue_count > 0:        
        return revenue_sum / revenue_count
    return 0    


df["AVG REVENUE"] = revenue_as_list.apply(lambda cell: calculate_avg_revenue(cell))


def calculate_avg_growth(cell):
    count = 0.0
    last_index = -1.0
    first_amount = 0.0
    last_amount = 0.0
    for index, amount in enumerate(cell):
        if amount > 0:
            if first_amount == 0:
                first_amount = amount
            if last_index > -1:
               count += index - last_index
            last_index = index    
            last_amount = amount        
    if count > 1:
        return (last_amount/first_amount)**(1/count)
    return 0



df["AVG REVENUE GROWTH"] = revenue_as_list.apply(lambda cell: calculate_avg_growth(cell))

employees_as_list = df["EMPLOYEES (2016,2017,2018,2019,2020,2021,2022)"].apply(lambda cell: convert_to_list_of_numbers(cell))
df["AVG EMPLOYEE GROWTH"] = employees_as_list.apply(lambda cell: calculate_avg_growth(cell))

df["AVG EMPLOYEE GROWTH"]


0       0.924211
1       0.000000
2       1.087380
3       1.092608
4       0.000000
          ...   
8889    0.000000
8890    0.000000
8891    0.000000
8892    0.000000
8893    0.000000
Name: AVG EMPLOYEE GROWTH, Length: 8894, dtype: float64

Create Valuation_to_funding_ratio column

In [24]:
valuation_to_funding_ratio = []

for index,valuation in enumerate(df['VALUATION AVG']):
    if valuation > 0:
        if df.loc[index, "FUNDING UP UNTIL VALUATION DATE"] > 0:
            valuation_to_funding_ratio.append(valuation / df.loc[index, "FUNDING UP UNTIL VALUATION DATE"])
        else:
            valuation_to_funding_ratio.append(valuation)
    else:
        valuation_to_funding_ratio.append(0)

df["VALUATION TO FUNDING RATIO"] = valuation_to_funding_ratio


df['VALUATION TO FUNDING RATIO']


0       0.000000
1       3.279013
2       4.889778
3       0.000000
4       0.000000
          ...   
8889    0.000000
8890    0.000000
8891    0.000000
8892    0.000000
8893    0.000000
Name: VALUATION TO FUNDING RATIO, Length: 8894, dtype: float64

### 1.6 Female in Founder Team

In [25]:


# Add one-hot columns if female, male founder in team. Add unknown and consider later if team is mixed 
# (as criteria for success maybe)

#df['FEMALE FOUNDER', 'MALE FOUNDER', 'UNKNOWN FOUNDER'] = False
df.loc[:,'FEMALE FOUNDER'] = False
df.loc[:,'MALE FOUNDER'] = False
df.loc[:, 'UNKNOWN FOUNDER'] = False


df.loc[df['FOUNDERS GENDERS'].str.contains('female' ,na= False), 'FEMALE FOUNDER'] = True
df.loc[df['FOUNDERS GENDERS'].str.contains('(?<!fe)male', na=False, regex=True), 'MALE FOUNDER'] = True
df.loc[df['FOUNDERS GENDERS'].str.contains('n/a', na = True), 'UNKNOWN FOUNDER'] = True

#df[['FOUNDERS GENDERS', 'FEMALE FOUNDER', 'MALE FOUNDER', 'UNKNOWN FOUNDER']]

df["MALE FOUNDER"]

0       False
1        True
2        True
3       False
4       False
        ...  
8889    False
8890    False
8891    False
8892    False
8893    False
Name: MALE FOUNDER, Length: 8894, dtype: bool

### 1.7 Founder is serial

In [26]:
df.loc[:,'FOUNDER SERIAL'] = False
df.loc[df['FOUNDERS IS SERIAL'].isnull(), 'FOUNDER SERIAL'] = 0.5
#df.loc[df['FOUNDERS IS SERIAL'].isnull(), 'FOUNDER SERIAL'] = 'Unknown'
df.loc[df['FOUNDERS IS SERIAL'].str.contains('yes' ,na= False), 'FOUNDER SERIAL'] = True

df['FOUNDER SERIAL']

0         0.5
1       False
2       False
3       False
4         0.5
        ...  
8889      0.5
8890      0.5
8891      0.5
8892      0.5
8893      0.5
Name: FOUNDER SERIAL, Length: 8894, dtype: object

### 1.8 MIXED TEAM

In [27]:
df['MIXED TEAM'] = 'Unknown'

#indexMix = df[(df['FEMALE FOUNDER'] > 0) & (df['MALE FOUNDER'] > 0)].index

# A mixed team has female founder and male founder 

df.loc[(df['FEMALE FOUNDER'] ==1 ) & (df['MALE FOUNDER'] ==1 ), 'MIXED TEAM'] = True
df.loc[((df['FEMALE FOUNDER'] ==1 ) ^ (df['MALE FOUNDER'] ==1) & (df['UNKNOWN FOUNDER'] ==0) ), 'MIXED TEAM'] = False


###  Create CITY COLUMNS

In [28]:
cities = set([])
df["HQ CITY"]= df["HQ CITY"].apply(lambda city: city.upper() if isinstance(city,str) else city)
for city in df["HQ CITY"]:
   cities.add(city)


for index,city in enumerate(cities):
    df.loc[:,f'HQ {city}'] = False


for index,city in enumerate(df["HQ CITY"]):
    if isinstance(city, str):
        df.loc[index, f'HQ {city}'] = True




/home/markus/.cache/pypoetry/virtualenvs/jupyter-pandas-cmPRGyoq-py3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1667: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = value


###  Create Revenue model COLUMNS

In [29]:
revenue_models = set([])

for revenue_model in df["REVENUE MODEL"]:
    if isinstance(revenue_model,str):
        for model in revenue_model.split(";"):
            revenue_models.add(model.upper())


for index, revenue_model in enumerate(revenue_models):
    df.loc[:, f'REVENUE MODEL {revenue_model}'] = False


for index, revenue_model in enumerate(df["REVENUE MODEL"]):
    if isinstance(revenue_model, str):
        for model in revenue_model.split(";"):
            df.loc[index, f'REVENUE MODEL {model.upper()}'] = True



### Select Columns relevant for training and export to CSV

Boolean Columns with less than 5% True rate are being dropped

In [30]:
df_target = df[['INDEX','TOTAL FUNDING USD', 'FUNDING FROM EARLY STAGES', 'FUNDING FROM EARLY STAGES > 2M', 'FUNDING UP UNTIL VALUATION DATE', 'AVG REVENUE', 'AVG REVENUE GROWTH', 'AVG EMPLOYEE GROWTH', 'VALUATION TO FUNDING RATIO']
   + [f'IS_{industry}_INDUSTRY'for industry in industries.keys()]
   + [f'REVENUE MODEL {model}'for model in revenue_models]
   + [f'HQ {city}'for city in cities]]



for column in df_target:
   if df_target[column].dtype == 'bool':
       df_target[column] = df_target[column].astype(int)
       if df_target[column].sum() < 0.05*df_target.shape[0]:
           df_target.drop([column], axis=1,inplace=True)

df_target.to_csv("data_for_training.csv")

/tmp/ipykernel_936525/4168001108.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_target[column] = df_target[column].astype(int)
/home/markus/.cache/pypoetry/virtualenvs/jupyter-pandas-cmPRGyoq-py3.9/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
